In [5]:
# importing some useful libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns  
import time 

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from nltk.tokenize import RegexpTokenizer  
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import pickle 

# Wczytywanie danych

In [4]:
df= pd.read_csv("../Data/URLs-mixed/kaggle_labeled.csv")

# Trenowanie modelu

In [22]:
## RegexpTokenizer
tokenizer = RegexpTokenizer(r'[A-Za-z]+')
tokenizer.tokenize(df.url[0]) # this will fetch all the words from the first URL
# Tokenizing all the rows 
print('Getting words tokenized ...')
t0= time.perf_counter()
df['text_tokenized'] = df.url.map(lambda t: tokenizer.tokenize(t))
t1 = time.perf_counter() - t0
print('Time taken',t1 ,'sec')

## SnowballStemmer
stemmer = SnowballStemmer("english") # choose a language
# Getting all the stemmed words
df['text_stemmed'] = df['text_tokenized'].map(lambda l: [stemmer.stem(word) for word in l])

# Joining all the stemmmed words.
df['text_sent'] = df['text_stemmed'].map(lambda l: ' '.join(l))

## Creating model
cv = CountVectorizer()
feature = cv.fit_transform(df.text_sent) #transform all text which we tokenize and stemed

trainX, testX, trainY, testY = train_test_split(feature, df.label)

Getting words tokenized ...
Time taken 1.5430248999999776 sec


ValueError: Logistic Regression supports only penalties in ['l1', 'l2', 'elasticnet', 'none'], got saga.

In [24]:
## LogisticRegression
lr = LogisticRegression(solver='saga',max_iter=350)
lr.fit(trainX,trainY)

print("Score: ",lr.score(testX,testY))

Score:  0.967055443510027


c:\users\piotr\pycharmprojects\hackathon-2021-rtfd\venv\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [26]:
lr.predict(cv.transform([' '.join([stemmer.stem(word) for word in tokenizer.tokenize("olx.pl")])]))

array(['bad'], dtype=object)